In [1]:
print("Hello World")

Hello World


In [2]:
import pandas as pd
import pyodbc

In [3]:
# Connect to DataBank
connection_string = (
    "DSN=DataBank;"
    "Trusted_Connection=yes;"
)

con = pyodbc.connect(connection_string)

In [4]:
# Write sql query
qry = "SELECT * FROM ukri_ssd_snapshot.2024_10_04_ssd_award;"

# Read sql query
df= pd.read_sql(qry, con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_32620\4121153418.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df= pd.read_sql(qry, con)


In [5]:
df['award_reference'].isna().sum()

251

In [6]:
df['award_reference'] = df['award_reference'].replace('Unknown', None)


In [7]:
df[df['award_reference'].isna()]

print(df['award_reference'].isna().sum()/len(df)*100)

# Only represents 0.13% of awards

df[df['award_reference'].isna()]['funded_amount'].sum()/df['funded_amount'].sum()*100

# or 0.15% of total funded amount so we can neglect

0.1261505844657307


0.1517036963978507

In [8]:
df_dropna = df[~df['award_reference'].isna()]

In [9]:
# award_reference is not Null and is duplicated
df_dropna[df_dropna['award_reference'].duplicated(keep=False)].sort_values(by='award_reference')['funded_amount'].sum()/df['funded_amount'].sum()*100

0.3187965192515037

In [10]:
df_dropna[df_dropna['award_reference'].duplicated(keep=False)].sort_values(by='award_reference').to_clipboard(index=False)

In [11]:
# Those duplicated represent 0.32% of total funded amount so we neglect

df_dropna.drop_duplicates(subset='award_reference', keep=False, inplace=True)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_32620\1863951632.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropna.drop_duplicates(subset='award_reference', keep=False, inplace=True)


In [12]:
df_dropna[df_dropna['start_date'].isna() & df_dropna['end_date'].isna()]['funded_amount'].sum() / df['funded_amount'].sum()*100

1.3501916844453263

In [13]:
# Awards where there are no start or end date (so we cannot tell if they are active or not) represent 1.35% of the total funded value

In [14]:
df_dropna = df_dropna[~(df_dropna['start_date'].isna() & df_dropna['end_date'].isna())]


In [15]:
df_dropna[df_dropna['start_date'].isna()]

,application_reference,award_reference,funder,administrator,application_source,announced_start_date,start_date,end_date,funded_amount,profiled_amount,project_cost,termination_date


In [16]:
df_dropna[df_dropna['end_date'].isna()]['funded_amount'].sum() # Those with end date is NaN are 0 funded amount so can neglect

0.0

In [17]:
df_dropna = df_dropna[~df_dropna['end_date'].isna()]

In [18]:
df_dropna[df_dropna['start_date'] > df_dropna['end_date']]['funded_amount'].sum() / df['funded_amount'].sum()*100 # negligable

0.00099942482526111

In [19]:
df_dropna = df_dropna[df_dropna['start_date'] < df_dropna['end_date']]


In [20]:
df_dropna['funded_amount'].sum() / df['funded_amount'].sum()*100

98.15924984814613

In [21]:
def list_years(row):
    return list(range(row['start_year'], row['end_year'] + 1))

In [22]:
df_dropna['award_reference'].duplicated().sum(0)

data_table = df_dropna.copy()[['award_reference', 'start_date', 'end_date', 'funded_amount']]

data_table['start_year'] = data_table['start_date'].dt.year
data_table['end_year'] = data_table['end_date'].dt.year

data_table['number_years'] = data_table['end_year'] - data_table['start_year'] + 1

data_table['years_active'] = data_table.apply(list_years, axis=1)

data_table

,award_reference,start_date,end_date,funded_amount,start_year,end_year,number_years,years_active
0,MR/Z503976/1,2024-08-01,2026-01-31,154977.54,2024,2026,3,"[2024, 2025, 2026]"
1,BB/Y513878/1,2024-02-14,2025-05-13,236363.08,2024,2025,2,"[2024, 2025]"
2,EP/Z53285X/1,2024-09-02,2031-09-01,11857652.76,2024,2031,8,"[2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031]"
4,BB/Y513891/1,2024-02-14,2025-08-13,258239.42,2024,2025,2,"[2024, 2025]"
5,ST/Y510002/1,2024-04-01,2027-03-31,502072.99,2024,2027,4,"[2024, 2025, 2026, 2027]"
...,...,...,...,...,...,...,...,...
208476,CCF ST:London Social Venture Fund,2023-12-01,2025-11-01,1499500.00,2023,2025,3,"[2023, 2024, 2025]"
208477,CCF ST:Innovation Catalysed & Enabled (ICE) Le...,2023-12-01,2024-11-01,329292.00,2023,2024,2,"[2023, 2024]"
208478,CCF ST:AI SuperConnector,2024-01-01,2025-07-01,1444479.00,2024,2025,2,"[2024, 2025]"
208479,CCF ST:North West Cyber Security Connect for C...,2023-12-01,2025-11-01,1230917.00,2023,2025,3,"[2023, 2024, 2025]"


In [23]:
data_table.sort_values(by='start_year')

,award_reference,start_date,end_date,funded_amount,start_year,end_year,number_years,years_active
164822,MC_U153026901,1951-01-01,1994-03-31,NaN,1951,1994,44,"[1951, 1952, 1953, 1954, 1955, 1956, 1957, 195..."
164200,MC_U147526898,1951-01-01,1994-03-31,NaN,1951,1994,44,"[1951, 1952, 1953, 1954, 1955, 1956, 1957, 195..."
13382,G5800055/1,1958-10-15,1994-04-30,413712.000,1958,1994,37,"[1958, 1959, 1960, 1961, 1962, 1963, 1964, 196..."
164910,MC_U117532065,1963-03-01,1995-09-30,NaN,1963,1995,33,"[1963, 1964, 1965, 1966, 1967, 1968, 1969, 197..."
11608,G6401740/1,1964-10-01,1992-09-30,0.000,1964,1992,29,"[1964, 1965, 1966, 1967, 1968, 1969, 1970, 197..."
...,...,...,...,...,...,...,...,...
55070,NE/Y000811/1,2026-06-01,2029-05-31,429536.560,2026,2029,4,"[2026, 2027, 2028, 2029]"
143341,ST/Y004795/1,2026-04-01,2028-03-31,85594.198,2026,2028,3,"[2026, 2027, 2028]"
1090,MR/Z505031/1,2026-02-01,2029-01-31,379182.640,2026,2029,4,"[2026, 2027, 2028, 2029]"
54629,NE/Y001338/1,2026-06-01,2029-05-31,502872.598,2026,2029,4,"[2026, 2027, 2028, 2029]"


In [24]:
data_table['funded_amount'].isna().sum()

23185

In [25]:
(data_table['funded_amount'] == 0).sum()

1247

In [26]:
(23185+1247)/len(data_table)*100

11.924098079025457

In [27]:
data_table.dropna(subset='funded_amount', inplace=True)


In [28]:
data_table =  data_table.copy()[data_table['funded_amount'] != 0]

In [29]:
data_table.sort_values(by='number_years')

,award_reference,start_date,end_date,funded_amount,start_year,end_year,number_years,years_active
101937,ES/I900845/1,2011-04-01,2011-06-01,26841.56,2011,2011,1,[2011]
31534,GR/J94839/01,1994-08-17,1994-11-16,5000.00,1994,1994,1,[1994]
193070,73596,2020-06-01,2020-11-30,39990.00,2020,2020,1,[2020]
31538,GR/J94891/01,1994-06-01,1994-08-31,3600.00,1994,1994,1,[1994]
110812,AH/I026162/1,2011-02-14,2011-09-01,24416.00,2011,2011,1,[2011]
...,...,...,...,...,...,...,...,...
11610,G7103979/1,1971-10-01,1996-09-30,1648074.00,1971,1996,26,"[1971, 1972, 1973, 1974, 1975, 1976, 1977, 197..."
162080,MC_48701003,1982-08-01,2007-07-31,178360.00,1982,2007,26,"[1982, 1983, 1984, 1985, 1986, 1987, 1988, 198..."
14109,G6901864/1,1969-07-14,1994-07-13,673710.00,1969,1994,26,"[1969, 1970, 1971, 1972, 1973, 1974, 1975, 197..."
162095,MC_48701002,1982-08-01,2007-07-31,195155.00,1982,2007,26,"[1982, 1983, 1984, 1985, 1986, 1987, 1988, 198..."


In [30]:
data_table[data_table['end_year'] > 2015].sort_values(by='number_years')

,award_reference,start_date,end_date,funded_amount,start_year,end_year,number_years,years_active
173519,971448,2016-03-01,2016-11-30,149500.000,2016,2016,1,[2016]
167617,72442,2020-06-01,2020-11-30,226497.000,2020,2020,1,[2020]
194323,73451,2020-06-01,2020-11-30,19053.000,2020,2020,1,[2020]
194316,56407,2020-06-01,2020-08-31,47242.000,2020,2020,1,[2020]
48404,BB/R005311/1,2017-06-05,2017-07-04,2310.000,2017,2017,1,[2017]
...,...,...,...,...,...,...,...,...
134251,NE/K010700/1,2013-09-01,2027-07-01,1018974.506,2013,2027,15,"[2013, 2014, 2015, 2016, 2017, 2018, 2019, 202..."
160476,MC_qA137916,2003-04-01,2017-12-31,150000.000,2003,2017,15,"[2003, 2004, 2005, 2006, 2007, 2008, 2009, 201..."
11393,G1100525/1,2010-10-01,2025-09-30,825455.000,2010,2025,16,"[2010, 2011, 2012, 2013, 2014, 2015, 2016, 201..."
124605,G1001147/1,2011-10-01,2027-03-31,4602857.200,2011,2027,17,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201..."


In [31]:
data_table['amount_per_year'] = data_table['funded_amount']/data_table['number_years']

df_explode = data_table.explode(column='years_active')

df_explode.groupby('years_active').mean(numeric_only=True)

,funded_amount,start_year,end_year,number_years,amount_per_year
years_active,,,,,
1958,4.137120e+05,1958.000000,1994.000000,37.000000,11181.405405
1959,4.137120e+05,1958.000000,1994.000000,37.000000,11181.405405
1960,4.137120e+05,1958.000000,1994.000000,37.000000,11181.405405
1961,4.137120e+05,1958.000000,1994.000000,37.000000,11181.405405
1962,4.137120e+05,1958.000000,1994.000000,37.000000,11181.405405
...,...,...,...,...,...
2029,4.353030e+06,2023.778614,2029.621988,6.843373,602750.518963
2030,6.932060e+06,2023.869565,2031.244565,8.375000,825720.017688
2031,8.681270e+06,2023.823009,2032.026549,9.203540,981883.252826


In [32]:
# So in total we can look at 98.2% of the total funded value, 1.8% is excluded

from datetime import date

today = date.today()
print("Today's date:", today)

# Active grants - start_date <= today and end_date >= today

data_ = df_dropna.copy()

data_['Active'] = 'Active'

data_.loc[data_.start_date > pd.to_datetime(today), 'Active'] = 'Future'

data_.loc[data_.end_date < pd.to_datetime(today), 'Active'] = 'Past'

Today's date: 2025-02-12


In [33]:
df_application = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2024_10_04_ssd_application;", con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_32620\3848588355.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_application = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2024_10_04_ssd_application;", con)


In [37]:
df_application['application_category'].unique()
df_application.columns

df_application['opportunity_name'].unique()

array(['New Generation Thinkers 2022',
       'MRC Infections and immunity Sep 2023: responsive mode: research grant',
       'EPSRC open fellowship May 2023', ..., 'UKRPIF: Net Zero',
       'Improve access and participation for Black, Asian and minority ethnic groups in postgraduate research study',
       'CCF RED'], dtype=object)

In [38]:
df_application[['opportunity_id', 'opportunity_name']].drop_duplicates()

,opportunity_id,opportunity_name
0,34,New Generation Thinkers 2022
1,171,MRC Infections and immunity Sep 2023: responsi...
2,139,EPSRC open fellowship May 2023
3,147,ESRC responsive mode: secondary data analysis ...
5,156,MRC: Public Health Intervention Development (P...
...,...,...
575917,UKRPIF7,UKRPIF
575918,CCF ST,CCF RED
575923,RED28,RED Fund
575928,RED30,RED Fund


In [39]:
dim_opportunity = pd.read_sql("SELECT * FROM databank.dim_opportunity;", con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_32620\1554799385.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_opportunity = pd.read_sql("SELECT * FROM databank.dim_opportunity;", con)


In [42]:
dim_opportunity[['OpportunityName', 'OpportunityNumber', 'OpportunityOriginSource']].duplicated().sum()

1

In [44]:
dim_opportunity[dim_opportunity[['OpportunityName', 'OpportunityNumber', 'OpportunityOriginSource']].duplicated(keep=False)]

,OpportunitySKey,OpportunityName,OpportunityNumber,OpportunityStartDate,OpportunityEndDate,OpportunityCloseDate,OpportunityMode_AHRC,OpportunityOriginSource,OpportunityComments,ApplicantLedOrTargeted,IsResponsive
10079,3517200,Accelerating Innovation in Rail Round 5 (12 Mo...,61,2017-09-18 00:00:00,None,2017-11-15 12:00:00,None,IUK,None,Unassigned,Unassigned
10252,3517373,Accelerating Innovation in Rail Round 5 (12 Mo...,61,2017-09-18 00:00:00,None,2017-11-15 12:00:00,None,IUK,None,Unassigned,Unassigned


In [ ]:

data_application = df_application[['application_reference', 'application_source', 'application_title', 'decision_owner', 
                                   'application_category', 'lead_organisation_id', 'organisation_source']]

In [ ]:
print(data_application[['application_reference', 'application_source']].duplicated().sum())

print(data_[['application_reference', 'application_source']].duplicated().sum())
# application_reference and application_source are duplicated

In [ ]:
df_application['award_reference'] = df_application['award_reference'].replace('Unknown', None) # replace Unknown with NaN

In [ ]:
len(data_[data_[['application_reference', 'application_source']].duplicated(keep=False)].sort_values(by='application_reference'))

num = data_[data_[['application_reference', 'application_source']].duplicated(keep=False)].sort_values(by='application_reference')['funded_amount'].sum()

num / df['funded_amount'].sum()*100 # negligable

In [ ]:
data_.drop_duplicates(subset=['application_reference', 'application_source'], keep=False, inplace=True)

In [ ]:
data_merge = pd.merge(data_, data_application.drop_duplicates(subset=['application_reference', 'application_source'], keep=False), 
                      on=['application_reference', 'application_source'], how='left', indicator=True)

In [ ]:
data_merge[data_merge['_merge'] == 'left_only']['funded_amount'].sum() / df['funded_amount'].sum()*100 # negligable

In [ ]:
print(len(data_merge))
print(len(data_))

In [ ]:
data_both = data_merge[data_merge['_merge'] == 'both']

In [ ]:
data_both.drop(columns=['announced_start_date', 'termination_date', '_merge'], inplace=True)

In [ ]:
ssd_organisation = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2024_10_04_ssd_organisation;", con)

In [ ]:
data_both[['application_reference', 'application_source']].duplicated().sum()

In [ ]:
data_both['funded_amount'].sum() / df['funded_amount'].sum() # currently looking at 98% of the total funded amount 2% is excluded

In [ ]:
ssd_organisation[['organisation_id', 'organisation_source']].duplicated().sum()

In [ ]:
data_both.columns

In [ ]:
ssd_organisation.columns

In [ ]:
data_merge = pd.merge(data_both, ssd_organisation[['organisation_id', 'organisation_name', 'ukprn', 'organisation_source']], 
                      left_on=['lead_organisation_id', 'organisation_source'], 
                      right_on=['organisation_id', 'organisation_source'], 
                      how='left', indicator=True)

In [ ]:
print(len(data_merge))
print(len(data_both))


In [ ]:
data_merge.dtypes

data_merge['application_reference_source'] = data_merge['application_reference'] + '_' + data_merge['application_source']

In [ ]:
data_merge[data_merge['_merge'] == 'left_only'] # these don't have an organisation listed

data_both = data_merge[data_merge['_merge'] == 'both']

In [ ]:
# look at TRAC providers by TRAC group
path = "C:\\Users\\MHil02\\OneDrive - UKRI\\Research Financial Sustainability\\TRAC data\\raw_TRAC"
trac_providers = pd.read_excel(path[:-8] + 'trac_providers.xlsx', header=2)


In [ ]:
data_merge[data_merge['_merge'] == 'left_only'] # these don't have an organisation listed

data_left = data_merge[data_merge['_merge'] == 'left_only']

In [ ]:
ssd_organisation[ssd_organisation['organisation_id'] == '10007855']

data_left = data_left.dropna(how='all', axis=1).drop(columns='_merge')

In [ ]:
data_merge = pd.merge(data_left, ssd_organisation[['organisation_id', 'organisation_name', 'ukprn', 'organisation_source']],
                      left_on='lead_organisation_id', right_on='organisation_id') # join remaining on organisation id

In [ ]:
print(len(data_merge))
print(len(data_left))

In [ ]:
data_both

data_merge = data_merge.drop(columns='organisation_source_x').rename(columns={'organisation_source_y':'organisation_source'})


In [ ]:
data_merge['ukprn'] = data_merge['organisation_id']


In [ ]:
data_concat = pd.concat([data_both, data_merge])

In [ ]:
data_concat.drop(columns=['profiled_amount', 'project_cost', 'lead_organisation_id', '_merge'], inplace=True)

In [ ]:
data_concat['funded_amount'].sum() / df['funded_amount'].sum()*100

In [ ]:
trac_providers['UKPRN'] = trac_providers['UKPRN'].astype(str)

In [ ]:
data_concat[data_concat['organisation_name'].str.contains('Dundee')]['ukprn'].unique()

data_concat[data_concat['organisation_name'] == 'University of Dundee'] # 1347
data_concat[data_concat['ukprn'] == '10007852'] # 1347

data_dundee = data_concat.copy()[data_concat['organisation_name'] == 'University of Dundee']

data_dundee['application_reference_source'].duplicated().sum()

councils = ['AHRC', 'BBSRC', 'EPSRC', 'ESRC', 'MRC', 'NERC', 'RE', 'STFC', 'HEG', 'All']

data_dundee['council'] = data_dundee['funder']

data_dundee.loc[~data_dundee['council'].isin(councils), 'council'] = 'Other'

data_dundee_group = data_dundee.groupby(['council', 'Active', 'organisation_name'], as_index=False).agg({'funded_amount':'sum', 
                                                                                                         'application_reference_source':'nunique'})

data_dundee1 = data_dundee_group.copy()[data_dundee_group['Active'] != 'Past']

data_dundee1.loc['Total'] = data_dundee1.sum(numeric_only=True)

data_dundee1.to_clipboard(index=False)

data_dundee[data_dundee['Active'] != 'Past'][['council', 'funded_amount', 'Active', 'application_title', 'end_date']]

print(len(data_concat))

data_concat['funded_amount'].isna().sum() / len(data_concat) *100

In [ ]:
data_dundee[data_dundee['Active'] != 'Past'][['award_reference', 'council', 'funded_amount', 'Active', 'application_title', 'end_date']].sort_values(by='funded_amount', ascending=False).to_clipboard(index=False)


In [ ]:
# Ulster and Aberystwyth put in ukprns
data_concat.loc[data_concat['organisation_name'].str.contains('Ulster') & data_concat['organisation_name'].str.contains('University'), 'ukprn'] = '10007807'

data_concat.loc[data_concat['organisation_name'].str.contains('Aberystwyth'), 'ukprn'] = '10007856'

In [ ]:
data_ = pd.merge(data_concat, trac_providers[['UKPRN', 'Institution Name', 'TRAC Peer Group']], left_on='ukprn', right_on='UKPRN',
                 how='outer', indicator=True)

In [ ]:
data_left = data_[data_['_merge'] == 'left_only'].dropna(how='all', axis=1)

data_[data_['_merge'] == 'right_only'].dropna(how='all', axis=1)
# these are still missing but TRAC Bs are covered

In [ ]:
data_both = data_[data_['_merge'] == 'both']

In [ ]:
data_check = data_both[['organisation_name', 'ukprn']].drop_duplicates()

data_check[data_check['ukprn'].duplicated(keep=False)].to_clipboard(index=False) # checked

In [ ]:
trac_providers[trac_providers['TRAC Peer Group'] == 'B'] # quick check and most of these seem to be captured 

# look at TRAC B universities, if want to look at other TRAC groups need to check names and UKPRNs in data

data_b = data_both[data_both['TRAC Peer Group'] == 'B']

data_group = data_b.groupby(['funder', 'Active', 'Institution Name'], as_index=False).agg({'funded_amount':'sum','application_reference_source':'nunique'})

In [ ]:
data_group.to_clipboard(index=False)

In [ ]:
data_tmp = data_group.groupby(['Active', 'Institution Name'], as_index=False).sum(numeric_only=True)
data_tmp['funder'] = 'All'

data_out = pd.concat([data_group, data_tmp])

In [ ]:
data_out.to_clipboard(index=False)

In [ ]:
data_out

In [ ]:
data_out.groupby(['funder', 'Active'], as_index=False).quantile(0.5, numeric_only=True)

data_av = data_out[(data_out['funder'] == 'All') & (data_out['Active'] == 'Active')]

low = data_av['funded_amount'].quantile(0.25)
upp = data_av['funded_amount'].quantile(0.75)

data_av['Quantile'] = 'Middle'

data_av.loc[data_av['funded_amount'] < low, 'Quantile'] = 'Lower'
data_av.loc[data_av['funded_amount'] > upp, 'Quantile'] = 'Upper'

In [ ]:
unis = data_av.sort_values(by='funded_amount').loc[data_av['Quantile'] == 'Middle'][['Institution Name']]

In [ ]:
unis

In [ ]:
data_merge = pd.merge(data_out, unis, on='Institution Name')

In [ ]:
data_merge['funder'].unique()

In [ ]:
councils = ['AHRC', 'BBSRC', 'EPSRC', 'ESRC', 'MRC', 'NERC', 'RE', 'STFC', 'HEG', 'All']

data_merge.loc[~data_merge['funder'].isin(councils), 'funder'] = 'Other'

In [ ]:
# Studentships

data_students = pd.read_excel("C:\\Users\\MHil02\\UKRI\\RICN - UKRI Research & Innovation Careers Network - 01_Core_talent_tables\\UKRI studentship counts and expenditure (Oct 24 SSD snapshot).xlsx", sheet_name='FY Organisation')

In [ ]:
data_2425 = data_students[data_students['Financial Year'] == '2023-24']

data_2425['Council'].unique()

In [ ]:
# active lead org
data_select = data_2425.copy()[['Council', 'Organisation', 'active_host_org']]
data_select['Council'] = data_select['Council'].replace('All UKRI', 'All')

data_select.loc[~data_select['Council'].isin(councils), 'Council'] = 'Other'

In [ ]:
data_select['Organisation'].isna().sum()

data_select[data_select['Organisation'].isna()]

data_dundee = data_select.dropna(subset='Organisation')

data_dundee = data_dundee[data_dundee['Organisation'].str.contains('Dundee')]

data = pd.concat([data_dundee[~data_dundee['Council'].isin(['All', 'Other'])],
           data_dundee[data_dundee['Council'] == 'Other'],
           data_dundee[data_dundee['Council'] == 'All']])

data.to_clipboard(index=False)

In [ ]:
data_select

data1 = data_merge.loc[data_merge['Active'] != 'Past'].groupby(['funder', 'Active', 'Institution Name'], as_index=False).sum()

In [ ]:
data2 = data_select.groupby(['Council', 'Organisation'], as_index=False).sum()


In [ ]:
data1.columns

In [ ]:
data1['Institution Name'].unique()

In [ ]:
data2['Organisation'].unique()

data2[data2['Organisation'].str.contains('Brunel ')]
data2.rename(columns={'Organisation':'Institution Name'}, inplace=True)

In [ ]:
# Replace Aberystwyth
data1.loc[data1['Institution Name'].str.contains('Aberystwyth'), 'Institution Name'] = 'Aberystwyth University'
data2.loc[data2['Institution Name'].str.contains('Economics'), 'Institution Name'] = 'The London School of Economics and Political Science'
data2.loc[data2['Institution Name'].str.contains('East Anglia'), 'Institution Name'] = 'The University of East Anglia'
data2.loc[data2['Institution Name'].str.contains('Kent'), 'Institution Name'] = 'The University of Kent'
data2.loc[data2['Institution Name'].str.contains('Reading'), 'Institution Name'] = 'The University of Reading'
data2.loc[data2['Institution Name'].str.contains('Surrey'), 'Institution Name'] = 'The University of Surrey'


In [ ]:
data1.rename(columns={'funder':'Council'}, inplace=True)

In [ ]:
data1 = data1.rename(columns={'funded_amount':'Funded Amount', 'application_reference_source':'Number'})


In [ ]:

data2.rename(columns={'active_host_org':'Studentships'}, inplace=True)
data2['Active'] = 'Active'

In [ ]:
# Select Institutions
data2 = pd.merge(data2, unis, on='Institution Name')


In [ ]:
data_merge = pd.merge(data1, data2, on=['Active', 'Council', 'Institution Name'], how='outer', indicator=True)

In [ ]:
data_merge[data_merge['_merge'] == 'left_only']

data_merge[data_merge['_merge'] == 'right_only'].loc[data_merge['Institution Name'].str.contains('Stirling')]


In [ ]:
data_merge.dtypes
data_merge.drop(columns='_merge', inplace=True)
data_merge.fillna(0, inplace=True)

In [ ]:
data_out = data_merge.groupby(['Council', 'Active'], as_index=False).mean(numeric_only=True)


In [ ]:
data_sort = data_out.sort_values(by=['Council', 'Active'])

data = pd.concat([data_sort[~data_sort['Council'].isin(['All', 'Other'])],
           data_sort[data_sort['Council'] == 'Other'],
           data_sort[data_sort['Council'] == 'All']])


In [ ]:
data.to_clipboard(index=False)

In [ ]:
data_merge.loc[data_merge['Council'] == 'AHRC'].sum(numeric_only=True)

In [ ]:
data_merge.loc[data_merge['Council'] == 'AHRC']

In [ ]:
198/11

In [ ]:
data